***
# Screener

Operation:
1. Define function ```buysig()``` which scrapes stock data, calculates buy signals and returns pandas series holding information when the last buy signal happend and distance from today
2. Filter results to show only buy signals from last two days
3. Saves results into excel sheet

In [1]:
# imports
from scraper import stock_daily
from analyzer import Analyzer
from IPython.display import display
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import indicators as ind

***
## Define ```buysig()```

* Input: empty pandas series with predefined columns
* Output: said series with filled results

Scrapes data, calculate indicators and buy signals (depending on strategy), saves results into series and outputs it

In [2]:
def buysig(result):
    # SCRAPING
    try:
        stock = stock_daily(result["ticker"], save=False)
    except:
        print(result["ticker"] + ": Exception occured during data scraping, skipped.")
        return result
    
    # CREATING BUY SIGNALS
    # calculating indicators
    vfi = ind.vfi(stock.data, period=30, coef=0.2, vcoef=1.5)
    fs = ind.stoch(stock.data, period=5, sk=2, sd=3)
    ss = ind.stoch(stock.data, period=21, sk=2, sd=5)
    # calculating VFI histogram trend
    window = 3
    vfi_hist = vfi["histogram"].rolling(window=window).apply(lambda x: np.polyfit(np.arange(window), x, 1)[0], raw=True).values
    vfi_hist = vfi_hist > 0
    # VFI trend
    vfi_trend = vfi["vfi"] > vfi["vfi_smooth"]
    vfi_conf = np.logical_or(vfi_hist, vfi_trend)
    # calculating fast stochastic trend
    window = 4
    fs_conf = fs["k"].rolling(window=window).apply(lambda x: np.polyfit(np.arange(window), x, 1)[0], raw=True).values
    fs_conf = fs_conf > 0
    # buy signals
    conditions = np.logical_and((ss["k"] > ss["d"]).to_numpy(), (ss["d"] >= 0).to_numpy())
    bss = np.concatenate((np.array([0]), (conditions[:-1] < conditions[1:]))).astype("int")
    # finalize buy signals
    bsf = np.logical_and(np.logical_and(fs_conf, vfi_conf), bss)
    if bsf.sum() == 0:
        print(result["ticker"] + ": No buying signals generated, skipped.")
        return result
    
    # SAVING DATA
    # date of last buy signal and distance from today
    try:
        lastindex = np.squeeze(np.where(bsf == True))[-1]
        result["buy_date"] = stock.data.loc[lastindex,"Date"]
        result["distance"] = stock.data.index[-1] - lastindex
    except:
        print(result["ticker"] + "Something broke during saving, skipped.")
        return result

    return result

***
## Run the analysis

Tickers are stored in ```tickers.xlsx``` excel file where multiple sheets are defined. By modifying the *sheets* variable, one can choose which stocks to scan through

Sheets in the excel:
1. spy - holds all stock tickers present in the SPY ETF
2. iwm - holds top 600 holdings in the IWM ETF which follows Russell 2000
3. watchlist - list of stocks I'm personally interested in and found somewhere else, on fintwit, reddit, my own research
4. longs - stocks I have high conviction in and want to hold long term

Tickers can repeat, but it would be better to avoid it. Currently, scan through all stocks runs for ~15 minutes

In [3]:
# which sheets to use:
sheets = ["spy", "iwm", "watchlist", "longs", "midcap", "largecap"] # specific scan
# sheets = ["nasdaq"] # full nasdaq

# read tickers from excel file
excel = pd.read_excel("tickers.xlsx", sheet_name=sheets)

In [4]:
# define empty dataframe
data = pd.DataFrame(columns=["ticker", "type", "buy_date", "distance"])
# run analysis
for i in sheets:
    for j in excel[i]["ticker"].to_list():
        # define series
        ser = {"ticker": j, "type": i, "buy_date": np.nan, "distance": np.nan}
        ser = pd.Series(data=ser)

        res = buysig(ser).to_frame().T
        data = data.append(res, ignore_index=True)

- BRK.B: No data found, symbol may be delisted
BRK.B: No buying signals generated, skipped.
BBWI: No buying signals generated, skipped.
- BF.B: No data found for this date range, symbol may be delisted
BF.B: No buying signals generated, skipped.
OGNSomething broke during saving, skipped.
- XTSLA: No data found, symbol may be delisted
XTSLA: No buying signals generated, skipped.
DBRGSomething broke during saving, skipped.
MNTVSomething broke during saving, skipped.
- JWA: No data found, symbol may be delisted
JWA: No buying signals generated, skipped.
- MOGA: No data found, symbol may be delisted
MOGA: No buying signals generated, skipped.
GMTX: No buying signals generated, skipped.
DBGISomething broke during saving, skipped.
SMFR: No buying signals generated, skipped.
GLBESomething broke during saving, skipped.
MTTR: No buying signals generated, skipped.
GRPH: No buying signals generated, skipped.
UK: No buying signals generated, skipped.
OEG: No buying signals generated, skipped.
SLI:

***
## Print & Save results

In [5]:
# display results
display(data[data["distance"] <= 1].drop_duplicates(subset="ticker").reset_index(drop=True))
# save data
data[data["distance"] <= 5].drop_duplicates(subset="ticker").reset_index(drop=True).to_excel("Screener-"+str(dt.date.today())+".xlsx")

,ticker,type,buy_date,distance
0,NKE,spy,2021-09-09,1
1,WFC,spy,2021-09-10,0
2,TXN,spy,2021-09-10,0
3,UNP,spy,2021-09-09,1
4,ADP,spy,2021-09-09,1
...,...,...,...,...
106,TWLO,largecap,2021-09-03,1
107,WMG,largecap,2021-09-10,0
108,YNDX,largecap,2021-09-10,0
109,ZBRA,largecap,2021-09-03,1
